In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv(r'C:\Users\pavan\Desktop\data science\column_transformer _imputer _pipeline\telco_data.csv')

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1.0,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34.0,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2.0,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45.0,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2.0,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            6293 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           6043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            4543 non-null   float64
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   6043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       5543 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df['SeniorCitizen'].value_counts()

SeniorCitizen
0    5901
1    1142
Name: count, dtype: int64

In [6]:
for col in df.columns:
    if df[col].isnull().any():
        print(col,'-->',df[col].isnull().sum())

gender --> 750
Partner --> 1000
tenure --> 2500
InternetService --> 1000
StreamingTV --> 1500
MonthlyCharges --> 1500


In [7]:
df['TotalCharges']=df['TotalCharges'].str.strip()
df['TotalCharges']=df['TotalCharges'].replace('',None)
df['TotalCharges']=df['TotalCharges'].astype(float)

In [8]:
x=df.drop(columns=['Churn','customerID'])
y=df['Churn']

In [9]:
num_colms=x.select_dtypes(include='number').columns
cat_colms=x.select_dtypes(exclude='number').columns

# Simple Imputer

It replaces the missing values with the help of strategy(mean,median,most_frequent) or constant with fill_value

it accepts only 2D

In [10]:
# preprocessing=ColumnTransformer(transformers=[

# ('imputer_for_num_cols',SimpleImputer(strategy='mean'),num_colms.columns),
# ('scaling-for_num_cols',StandardScaler(),num_colms.columns)

# ])

In [11]:
num_pipelines=Pipeline(steps=[
('imputer_for_num_cols',SimpleImputer(strategy='mean')),
('scaling-for_num_cols',StandardScaler())
])


In [12]:
cat_pipelines=Pipeline(steps=[
('imputer_for_cat_cols',SimpleImputer(strategy='constant',fill_value='Unknown')),
('encoding_for_cat_cols',OrdinalEncoder())
])


In [13]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num_pipelines',num_pipelines,num_colms),
        ('cat_pipelines',cat_pipelines,cat_colms)
    ]
)


In [14]:
main_pipeline=Pipeline(
    steps=[
        ('preprocessing',preprocessing),
        ('model',LogisticRegression())
    ]
)

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [16]:
grid=GridSearchCV(

    estimator=main_pipeline,
    param_grid={
        'model__C':[0.01,0.1,1.0,10],
        'model__penalty':['l1','l2'],
        'model__solver':['liblinear'],
        'model__class_weight':['balanced',None]
    },
    cv=5,
    n_jobs=-1,
    verbose=0,
    scoring='f1_macro'
)

In [17]:
main_pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipelines',
                                                  Pipeline(steps=[('imputer_for_num_cols',
                                                                   SimpleImputer()),
                                                                  ('scaling-for_num_cols',
                                                                   StandardScaler())]),
                                                  Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')),
                                                 ('cat_pipelines',
                                                  Pipeline(steps=[('imputer_for_cat_cols',
                                                                   SimpleImputer(fill_value='Un...
                                                                                 strategy='constant')),
                                                                  ('encoding_for_cat_cols',
                                                                   OrdinalEncoder())]),
                                                  Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object'))])),
                ('model', LogisticRegression())])

In [19]:
main_pipeline.predict(x_test)

ValueError: Found unknown categories ['Unknown'] in column 0 during transform